In [ ]:
import sqlalchemy as sql
import random
import pandas as pd
from bokeh.layouts import column
from xarray.util.generate_ops import inplace

data = pd.read_excel('./data.xlsx')
backup = data.copy()
data = backup.copy()

In [ ]:
# Lista de cidades do RJ
cidades_rj = [
    "Rio de Janeiro", "Niterói", "Petrópolis", "Nova Friburgo", "Teresópolis",
    "Volta Redonda", "Resende", "Barra Mansa", "Macaé", "Campos dos Goytacazes",
    "Angra dos Reis", "Cabo Frio", "Araruama", "São Gonçalo", "Itaboraí",
    "Magé", "Queimados", "Belford Roxo", "Duque de Caxias", "Nova Iguaçu"
]

# Lista de cidades de SP
cidades_sp = [
    "São Paulo", "Campinas", "Santos", "São Bernardo do Campo", "São José dos Campos",
    "Ribeirão Preto", "Sorocaba", "Santo André", "Osasco", "Jundiaí",
    "Bauru", "Piracicaba", "Franca", "Guarulhos", "Diadema",
    "Barueri", "Taboão da Serra", "Mogi das Cruzes", "Taubaté", "Suzano"
]

todas_cidades = cidades_rj + cidades_sp

estados = ['São Paulo', 'Rio de Janeiro']

# inserindo todas as cidades de SP e RJ na coluna 'Cidade'
data['Cidade'] = [random.choice(todas_cidades) for _ in range(data.shape[0])]
data['Estado'] = [random.choice(estados) for _ in range(data.shape[0])]
print(data)

In [ ]:
data['Sigla do estado'] = data['Estado'].apply(lambda x: 'SP' if x == 'São Paulo' else 'RJ')

In [ ]:
data['Idade'] = [random.randint(18, 60) for _ in range(data.shape[0])]

In [ ]:
names = [
    "Ana", "João", "Maria", "Pedro", "Lucas",
    "Beatriz", "Carlos", "Júlia", "Rafael", "Camila",
    "Gabriel", "Fernanda", "Matheus", "Larissa", "Bruno",
    "Mariana", "Henrique", "Luana", "Felipe", "Letícia"
]

data['Name'] = [random.choice(names) for _ in range(data.shape[0])]

In [181]:
# conectando em um banco de dados
engine = sql.create_engine('sqlite:///:memory:')
conn = engine.connect()
data.drop(columns=['Index'], inplace=True)
data.to_sql('clientes', conn, index=False, if_exists='replace')

242

In [ ]:
# função para consultas
def query(query, connection):
    query = pd.read_sql(query, connection)
    print(query)
    return query

In [ ]:
# Consultando quantidade de nomes repetidos
q = '''
    select name, count(name) as nomes_repetidos
    from clientes
    group by name
    having count(name) > 1;
'''
query(q, engine)

In [ ]:
q1 = '''
    select
        c."name",
        max(c."Saldo na conta") as Saldo_Maximo
    from clientes c;
'''

q_final = '''
    select
        c."name",
        c."Saldo na conta",
        c."Estado",
        case
            when c."Saldo na conta" > 70000 then 'Premium'
            when c."Saldo na conta" between 10000 and 70000 then 'Gold'
            else 'Regular'
        end as "nivel_cliente"
    from clientes c;
'''

new_data = pd.read_sql(q_final, engine)
data['nivel_cliente'] = new_data['nivel_cliente']

query(q_final, engine)

In [182]:
data.to_excel('./data.xlsx', index_label='Index')